In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import jaccard_score
import os
from PIL import Image
import pandas as pd

# Set up directories and image names
image_names = [
    'West_Highland_white_terrier', 'American_coot', 'racer', 'flamingo', 'kite',
    'goldfish', 'tiger_shark', 'vulture', 'common_iguana', 'orange'
]
gradcam_dir = '/content/heatmaps/gradcam/'
lime_dir = '/content/heatmaps/lime/'
output_dir = '/content/analysis_output/'
os.makedirs(output_dir, exist_ok=True)



In [ ]:
# Image complexity labels (animate vs inanimate)
image_complexity = {
    'West_Highland_white_terrier': 'animate',
    'American_coot': 'animate',
    'racer': 'inanimate',
    'flamingo': 'animate',
    'kite': 'inanimate',
    'goldfish': 'animate',
    'tiger_shark': 'animate',
    'vulture': 'animate',
    'common_iguana': 'animate',
    'orange': 'inanimate'
}



In [ ]:
# Debug: List directory contents
print("Grad-CAM directory contents:", os.listdir(gradcam_dir))
print("LIME directory contents:", os.listdir(lime_dir))

# Function to find file with case-insensitive matching
def find_heatmap_file(directory, base_name, extensions=['.png', '.jpg']):
    for ext in extensions:
        for file in os.listdir(directory):
            if file.lower() == f"{base_name.lower()}{ext}":
                return os.path.join(directory, file)
    return None

# Function to load and binarize heatmaps
def load_and_binarize_heatmap(filepath, threshold=0.5):
    try:
        heatmap = np.array(Image.open(filepath).convert('L')) / 255.0
        return (heatmap > threshold).astype(np.uint8)
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None

# Function to calculate IoU
def calculate_iou(heatmap1, heatmap2):
    return jaccard_score(heatmap1.flatten(), heatmap2.flatten())

# Initialize results
results = []



In [ ]:
# Process each image
for img_name in image_names:
    # Use base image name without suffixes
    gradcam_path = find_heatmap_file(gradcam_dir, img_name)
    lime_path = find_heatmap_file(lime_dir, img_name)

    print(f"\nProcessing {img_name}:")
    print(f"  Grad-CAM: {gradcam_path}")
    print(f"  LIME: {lime_path}")

    if not gradcam_path or not lime_path:
        print(f"Skipping {img_name}: File not found")
        results.append({'Image': img_name, 'IoU': 'N/A', 'Type': image_complexity[img_name],
                        'GradCAM_Time': 'N/A', 'LIME_Time': 'N/A',
                        'GradCAM_Fidelity': 'N/A', 'LIME_Fidelity': 'N/A'})
        continue

    gradcam_heatmap = load_and_binarize_heatmap(gradcam_path)
    lime_heatmap = load_and_binarize_heatmap(lime_path)

    if gradcam_heatmap is None or lime_heatmap is None:
        print(f"Skipping {img_name}: Failed to load heatmaps")
        results.append({'Image': img_name, 'IoU': 'N/A', 'Type': image_complexity[img_name],
                        'GradCAM_Time': 'N/A', 'LIME_Time': 'N/A',
                        'GradCAM_Fidelity': 'N/A', 'LIME_Fidelity': 'N/A'})
        continue

    # Calculate IoU
    iou = calculate_iou(gradcam_heatmap, lime_heatmap)

    # Placeholder execution time and fidelity
    gradcam_time = np.random.uniform(0.1, 1.0)
    lime_time = np.random.uniform(0.5, 2.0)
    gradcam_fidelity = np.random.uniform(0.7, 0.95)
    lime_fidelity = np.random.uniform(0.6, 0.9)

    results.append({
        'Image': img_name,
        'IoU': iou,
        'Type': image_complexity[img_name],
        'GradCAM_Time': gradcam_time,
        'LIME_Time': lime_time,
        'GradCAM_Fidelity': gradcam_fidelity,
        'LIME_Fidelity': lime_fidelity
    })

    # Visualize heatmaps
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.title(f'Grad-CAM: {img_name}')
    plt.imshow(gradcam_heatmap, cmap='jet')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.title(f'LIME: {img_name}')
    plt.imshow(lime_heatmap, cmap='jet')
    plt.axis('off')
    plt.suptitle(f'IoU: {iou:.3f}')
    plt.savefig(os.path.join(output_dir, f'{img_name}_comparison.png'))
    plt.close()



In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Analyze IoU by complexity
animate_ious = [r['IoU'] for r in results if r['Type'] == 'animate' and r['IoU'] != 'N/A']
inanimate_ious = [r['IoU'] for r in results if r['Type'] == 'inanimate' and r['IoU'] != 'N/A']
animate_iou_mean = np.mean(animate_ious) if animate_ious else 0
inanimate_iou_mean = np.mean(inanimate_ious) if inanimate_ious else 0

# Plot IoU distribution
plt.figure(figsize=(8, 6))
sns.boxplot(data=[animate_ious, inanimate_ious], palette='Set2')
plt.xticks([0, 1], ['Animate (e.g., Goldfish)', 'Inanimate (e.g., Kite)'])
plt.ylabel('IoU')
plt.title('IoU: Grad-CAM vs LIME by Image Type')
plt.savefig(os.path.join(output_dir, 'iou_by_type.png'))
plt.close()

# Plot execution time vs fidelity
plt.figure(figsize=(8, 6))
gradcam_times = [r['GradCAM_Time'] for r in results if r['GradCAM_Time'] != 'N/A']
gradcam_fidelities = [r['GradCAM_Fidelity'] for r in results if r['GradCAM_Fidelity'] != 'N/A']
lime_times = [r['LIME_Time'] for r in results if r['LIME_Time'] != 'N/A']
lime_fidelities = [r['LIME_Fidelity'] for r in results if r['LIME_Fidelity'] != 'N/A']
plt.scatter(gradcam_times, gradcam_fidelities, label='Grad-CAM', alpha=0.6, color='teal')
plt.scatter(lime_times, lime_fidelities, label='LIME', alpha=0.6, color='coral')
plt.xlabel('Execution Time (seconds)')
plt.ylabel('Fidelity')
plt.title('Execution Time vs Fidelity')
plt.legend()
plt.savefig(os.path.join(output_dir, 'execution_vs_fidelity.png'))
plt.close()

# Generate simple report
avg_iou = np.mean([r['IoU'] for r in results if r['IoU'] != 'N/A']) if any(r['IoU'] != 'N/A' for r in results) else 'N/A'
avg_gradcam_time = np.mean([r['GradCAM_Time'] for r in results if r['GradCAM_Time'] != 'N/A']) if any(r['GradCAM_Time'] != 'N/A' for r in results) else 'N/A'
avg_lime_time = np.mean([r['LIME_Time'] for r in results if r['LIME_Time'] != 'N/A']) if any(r['LIME_Time'] != 'N/A' for r in results) else 'N/A'
avg_gradcam_fidelity = np.mean([r['GradCAM_Fidelity'] for r in results if r['GradCAM_Fidelity'] != 'N/A']) if any(r['GradCAM_Fidelity'] != 'N/A' for r in results) else 'N/A'
avg_lime_fidelity = np.mean([r['LIME_Fidelity'] for r in results if r['LIME_Fidelity'] != 'N/A']) if any(r['LIME_Fidelity'] != 'N/A' for r in results) else 'N/A'
skipped_images = [r['Image'] for r in results if r['IoU'] == 'N/A']


results_df.to_csv(os.path.join(output_dir, 'results.csv'), index=False)
